In [1]:
using DifferentialEquations
using Plots
using Plots.PlotMeasures

In [ ]:
# Initial conditions function
function initial_conditions(x, y, Lx, Ly)
    if Ly-5 <= y <= Ly
        if abs(x - (Lx ÷ 2)) < Lx / 50
            n1 = 1.0  # Type 1
            n2 = 0.0
        else
            n1 = 0.0
            n2 = 1.0  # Type 2
        end
    else
        n1 = 0.0
        n2 = 0.0
    end
    return n1, n2
end

# Laplacian function
function laplacian(u, dx, dy)
    lx = (circshift(u, (0, 1)) .- 2u .+ circshift(u, (0, -1))) / dx^2
    ly = (circshift(u, (1, 0)) .- 2u .+ circshift(u, (-1, 0))) / dy^2

    lx[:, 1] .= (u[:, 2] .- u[:, 1]) / dx^2
    lx[:, end] .= (u[:, end-1] .- u[:, end]) / dx^2
    ly[1, :] .= (u[2, :] .- u[1, :]) / dy^2
    ly[end, :] .= (u[end-1, :] .- u[end, :]) / dy^2

    return lx + ly
end

function laplacian_x(u, dx, dy)
    lx = (circshift(u, (0, 1)) .- 2u .+ circshift(u, (0, -1))) / dx^2

    lx[:, 1] .= (u[:, 2] .- u[:, 1]) / dx^2
    lx[:, end] .= (u[:, end-1] .- u[:, end]) / dx^2

    return lx
end

function laplacian_y(u, dx, dy)
    ly = (circshift(u, (1, 0)) .- 2u .+ circshift(u, (-1, 0))) / dy^2

    ly[1, :] .= (u[2, :] .- u[1, :]) / dy^2
    ly[end, :] .= (u[end-1, :] .- u[end, :]) / dy^2

    return ly
end

# Partial derivatives functions
function ∂x(u, dx)
    du = (circshift(u, (0, -1)) .- circshift(u, (0, 1))) / (2dx)
    du[:, 1] .= (u[:, 2] .- u[:, 1]) / dx
    du[:, end] .= (u[:, end] .- u[:, end-1]) / dx
    return du
end

function ∂y(u, dy)
    du = (circshift(u, (-1, 0)) .- circshift(u, (1, 0))) / (2dy)
    du[1, :] .= (u[2, :] .- u[1, :]) / dy
    du[end, :] .= (u[end, :] .- u[end-1, :]) / dy
    return du
end

# Main simulation function
function simulate_reaction_diffusion(; r1=1.0, r2=1.0, K=0.1, D=0.0, D1x=0.01, D1y=0.01, D2x=0.01, D2y=0.01, a11 = 0.0, a21 = 0.0, a12 = 0.0, a22 = 0.0, 
                                     Lx=100.0, Ly=100.0, Nx=200, Ny=200, t_final = 100.0, num_times_saved=1)

    # Grid
    x = LinRange(0, Lx, Nx)
    y = LinRange(0, Ly, Ny)
    dx = x[2] - x[1]
    dy = y[2] - y[1]

    # Reaction-diffusion system function
    function reaction_diffusion!(du, u, p, t)
        n1 = u[1, :, :]
        n2 = u[2, :, :]
        n = n1 .+ n2
        f = n1 ./ (n .+ 1e-5)

        du[1, :, :] .= r1 .* (1 .+ a11 .* f .+ a12 .* (1 .- f)) .* n1 .* (1 .- n) .* (n .- K) .+ 
                       D1x .* laplacian_x(n1, dx, dy) .+ D1y .* laplacian_y(n1, dx, dy) 
                    #    D .* ∂x(n .* (1 .- n) .* ∂x(n1, dx), dx) + D .* ∂y(n .* (1 .- n) .* ∂y(n1, dy), dy)
                    # D .* (∂x(n,dx) .* ∂x(n1,dx) + n .* laplacian_x(n1,dx,dy) + ∂y(n,dy) .* ∂y(n1,dy) + n .* laplacian_y(n1,dx,dy)) 
        du[2, :, :] .= r2 .* (1 .+ a21 .* f .+ a22 .* (1 .- f)) .* n2 .* (1 .- n) .* (n .- K) .+  
                       D2x .* laplacian_x(n2, dx, dy) .+ D2y .* laplacian_y(n2, dx, dy) 
                    #    D .* ∂x(n .* ∂x(n2, dx), dx) + D .* ∂y(n .* ∂y(n2, dy), dy) 
                    #    D .* (∂x(n,dx) .* ∂x(n2,dx) + n .* laplacian_x(n2,dx,dy) + ∂y(n,dy) .* ∂y(n2,dy) + n .* laplacian_y(n2,dx,dy))
    end

    # Initialize conditions
    u0 = zeros(2, Ny, Nx)
    for i in 1:Nx
        for j in 1:Ny
            n1, n2 = initial_conditions(x[i], y[j], Lx, Ly)
            u0[1, j, i] = n1
            u0[2, j, i] = n2
        end
    end

    # PDE problem
    prob = ODEProblem(reaction_diffusion!, u0, (0.0, t_final))

    # Solve
    sol = solve(prob, Tsit5() ; saveat=t_final / num_times_saved)

    # Return the final solution
    return sol
end

using Colors

# Create the colormap
function create_colormap(f1, total_density)
    colormap = [RGB(f1[i, j], 1-f1[i, j], 0) * total_density[i, j] for i in 1:size(f1, 1), j in 1:size(f1, 2)]
    return colormap
end

function create_colormap(f1, total_density)
    light_sky_blue = RGB(0/255, 174/255, 239/255)
    yellow = RGB(249/255, 237/255, 50/255)
    colormap = [
        RGB(
            yellow.r * (1 - f1[i, j]) + light_sky_blue.r * f1[i, j],
            yellow.g * (1 - f1[i, j]) + light_sky_blue.g * f1[i, j],
            yellow.b * (1 - f1[i, j]) + light_sky_blue.b * f1[i, j]
        ) * total_density[i, j]
        for i in 1:size(f1, 1), j in 1:size(f1, 2)
    ]
    return colormap
end

function create_colormap(f1, total_density)
    light_sky_blue = RGB(0/255, 174/255, 239/255)
    yellow = RGB(249/255, 237/255, 50/255)
    white = RGB(1, 1, 1)  # Color for low-density regions

    colormap = [
        total_density[i, j] < 0.3 ? white :  # Set to white if density is low
        RGB(
            yellow.r * (1 - f1[i, j]) + light_sky_blue.r * f1[i, j],
            yellow.g * (1 - f1[i, j]) + light_sky_blue.g * f1[i, j],
            yellow.b * (1 - f1[i, j]) + light_sky_blue.b * f1[i, j]
        ) 
        for i in 1:size(f1, 1), j in 1:size(f1, 2)
    ]
    return colormap
end

using Colors

function create_colormap_binary(f1, total_density)
    light_sky_blue = RGB(0/255, 174/255, 239/255)
    yellow = RGB(249/255, 237/255, 50/255)
    white = RGB(1, 1, 1)  # Optional for unshaded regions

    colormap = [
        total_density[i, j] < 0.3 ? white : 
        (f1[i, j] > 0.52 ? light_sky_blue : f1[i, j] < 0.48 ? yellow : 0.5*(light_sky_blue + yellow))
        for i in 1:size(f1, 1), j in 1:size(f1, 2)
    ]
    return colormap
end


In [ ]:
solution = simulate_reaction_diffusion(r1=1.3, a21=-0.2, a12=-0.2,
                                        D1x=0.01, D2x=0.035, D1y=0.02, D2y=0.02,
                                        Lx=100.0, Ly=70.0, Nx=200, Ny=200, t_final=400.0)

sol = solution
# Calculate the fraction of type 1
f1 = sol[1, :, :, end] ./ (sol[1, :, :, end] .+ sol[2, :, :, end] .+ 1e-10)  # Add a small value to avoid division by zero

# Calculate the total density
total_density = sol[1, :, :, end] .+ sol[2, :, :, end]

# Generate the colormap
colormap = create_colormap(f1, total_density)

# Plot the heatmap
heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)                                        

In [ ]:
solution = simulate_reaction_diffusion(a11 = -0.7, a21=-0.7, a12=0.7, a22=0.0,
                                        D1x=0.02, D2x=0.09, D1y=0.02, D2y=0.02,
                                        Lx=100.0, Ly=70.0, Nx=300, Ny=300, t_final=400.0)

sol = solution
f1 = sol[1, :, :, end] ./ (sol[1, :, :, end] .+ sol[2, :, :, end] .+ 1e-10)  # Add a small value to avoid division by zero
total_density = sol[1, :, :, end] .+ sol[2, :, :, end]
colormap = create_colormap(f1, total_density)
# heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)
heatmap(colormap, axis=nothing, frame=:none, size=(800, 800), margins=0mm)

filename = "figures/dent_rd"
savefig(filename*".pdf")
savefig(filename*".svg")
savefig(filename*".png")

In [ ]:
solution = simulate_reaction_diffusion(a11 = 0.4, a21=0.4, a12=0.7+0.2, a22=0.0,
                                        D1x=0.02, D2x=0.09, D1y=0.02, D2y=0.02,
                                        Lx=100.0, Ly=70.0, Nx=300, Ny=300, t_final=400.0)

sol = solution
f1 = sol[1, :, :, end] ./ (sol[1, :, :, end] .+ sol[2, :, :, end] .+ 1e-10)  # Add a small value to avoid division by zero
total_density = sol[1, :, :, end] .+ sol[2, :, :, end]
colormap = create_colormap(f1, total_density)
# heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)
heatmap(colormap, axis=nothing, frame=:none, size=(800, 800), margins=0mm)

filename = "figures/composite_rd"
savefig(filename*".pdf")
savefig(filename*".svg")
savefig(filename*".png")

In [ ]:
solution = simulate_reaction_diffusion(a11 = 1.0, a21=1.0, a12=0.6, a22=0.0,
                                        D1x=0.02, D2x=0.09, D1y=0.02, D2y=0.02,
                                        Lx=100.0, Ly=70.0, Nx=300, Ny=300, t_final=400.0)

sol = solution
f1 = sol[1, :, :, end] ./ (sol[1, :, :, end] .+ sol[2, :, :, end] .+ 1e-10)  # Add a small value to avoid division by zero
total_density = sol[1, :, :, end] .+ sol[2, :, :, end]
colormap = create_colormap(f1, total_density)
# heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)
heatmap(colormap, axis=nothing, frame=:none, size=(800, 800), margins=0mm)

filename = "figures/circ_rd"
savefig(filename*".pdf")
savefig(filename*".svg")
savefig(filename*".png")

In [ ]:
solution = simulate_reaction_diffusion(a11 = 1.0, a21=1.0, a12=0.0, a22=0.0,
                                        D1x=0.02, D2x=0.09, D1y=0.02, D2y=0.02,
                                        Lx=100.0, Ly=70.0, Nx=300, Ny=300, t_final=400.0)

sol = solution
f1 = sol[1, :, :, end] ./ (sol[1, :, :, end] .+ sol[2, :, :, end] .+ 1e-10)  # Add a small value to avoid division by zero
total_density = sol[1, :, :, end] .+ sol[2, :, :, end]
colormap = create_colormap(f1, total_density)
# heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)
heatmap(colormap, axis=nothing, frame=:none, size=(800, 800), margins=0mm)

filename = "figures/escape_rd"
savefig(filename*".pdf")
savefig(filename*".svg")
savefig(filename*".png")

In [ ]:
function initial_conditions(x, y, Lx, Ly, n_wedges=12, radius_fraction=0.03)
    # Center of the grid
    center_x, center_y = Lx / 2, Ly / 2
    
    # Radius of the circle as a fraction of the grid size
    radius = radius_fraction * min(Lx, Ly)

    # Calculate distance and angle from the center
    dist_to_center = hypot(x - center_x, y - center_y)
    angle = atan(y - center_y, x - center_x)
    
    # Normalize angle to [0, 2π)
    angle = angle < 0 ? angle + 2π : angle

    # Determine wedge index
    wedge_index = Int(floor(angle / (2π / n_wedges)))

    # Inside the circle
    if dist_to_center <= radius
        # Alternate between type 1 and type 2 based on wedge index
        if isodd(wedge_index)
            n1, n2 = 1.0, 0.0  # Type 1
        else
            n1, n2 = 0.0, 1.0  # Type 2
        end
    else
        n1, n2 = 0.0, 0.0  # Outside the circle
    end
    
    return n1, n2
end


solution = simulate_reaction_diffusion(r1=3.0, r2=3.0, a21=0.0, a12=0.0,
                                        D1x=0.006, D2x=0.006, D1y=0.006, D2y=0.006,
                                        Lx=100.0, Ly=100.0, Nx=500, Ny=500, t_final=200.0)

sol = solution
f1 = sol[1, :, :, end] ./ (sol[1, :, :, end] .+ sol[2, :, :, end] .+ 1e-10)  # Add a small value to avoid division by zero
total_density = sol[1, :, :, end] .+ sol[2, :, :, end]
colormap = create_colormap_binary(f1, total_density)
# heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)
heatmap(colormap, axis=nothing, frame=:none, size=(800, 800), margins=0mm)

filename = "figures/neutral_iso"
savefig(filename*".pdf")
savefig(filename*".svg")
savefig(filename*".png")

In [ ]:
function initial_conditions(x, y, Lx, Ly, n_wedges=12, radius_fraction=0.03)
    # Center of the grid
    center_x, center_y = Lx / 2, Ly / 2
    
    # Radius of the circle as a fraction of the grid size
    radius = radius_fraction * min(Lx, Ly)

    # Calculate distance and angle from the center
    dist_to_center = hypot(x - center_x, y - center_y)
    angle = atan(y - center_y, x - center_x)
    
    # Normalize angle to [0, 2π)
    angle = angle < 0 ? angle + 2π : angle

    # Determine wedge index
    wedge_index = Int(floor(angle / (2π / n_wedges)))

    # Inside the circle
    if dist_to_center <= radius
        # Alternate between type 1 and type 2 based on wedge index
        if isodd(wedge_index)
            n1, n2 = 1.0, 0.0  # Type 1
        else
            n1, n2 = 0.0, 1.0  # Type 2
        end
    else
        n1, n2 = 0.0, 0.0  # Outside the circle
    end
    
    return n1, n2
end


solution = simulate_reaction_diffusion(r1=3.0, r2=3.0, a21=0.0, a12=0.0,
                                        D1x=0.006, D2x=0.006, D1y=0.003, D2y=0.003,
                                        Lx=100.0, Ly=100.0, Nx=500, Ny=500, t_final=200.0)

sol = solution
f1 = sol[1, :, :, end] ./ (sol[1, :, :, end] .+ sol[2, :, :, end] .+ 1e-10)  # Add a small value to avoid division by zero
total_density = sol[1, :, :, end] .+ sol[2, :, :, end]
colormap = create_colormap_binary(f1, total_density)
# heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)
heatmap(colormap, axis=nothing, frame=:none, size=(800, 800), margins=0mm)

filename = "figures/neutral_weak_aniso"
savefig(filename*".pdf")
savefig(filename*".svg")
savefig(filename*".png")

In [ ]:
function initial_conditions(x, y, Lx, Ly, n_wedges=8, radius_fraction=0.03)
    # Center of the grid
    center_x, center_y = Lx / 2, Ly / 2
    
    # Radius of the circle as a fraction of the grid size
    radius = radius_fraction * min(Lx, Ly)

    # Calculate distance and angle from the center
    dist_to_center = hypot(x - center_x, y - center_y)
    angle = atan(y - center_y, x - center_x)
    
    # Normalize angle to [0, 2π)
    angle = angle < 0 ? angle + 2π : angle

    # Determine wedge index
    wedge_index = Int(floor(angle / (2π / n_wedges)))

    # Inside the circle
    if dist_to_center <= radius
        # Alternate between type 1 and type 2 based on wedge index
        if isodd(wedge_index)
            n1, n2 = 1.0, 0.0  # Type 1
        else
            n1, n2 = 0.0, 1.0  # Type 2
        end
    else
        n1, n2 = 0.0, 0.0  # Outside the circle
    end
    
    return n1, n2
end


solution = simulate_reaction_diffusion(r1=3.0, r2=3.0, a21=0.0, a12=0.0,
                                        D1x=0.006, D2x=0.006, D1y=0.01, D2y=0.01, D = 0.01,
                                        Lx=100.0, Ly=100.0, Nx=500, Ny=500, t_final=200.0)

sol = solution
f1 = sol[1, :, :, end] ./ (sol[1, :, :, end] .+ sol[2, :, :, end] .+ 1e-10)  # Add a small value to avoid division by zero
total_density = sol[1, :, :, end] .+ sol[2, :, :, end]
colormap = create_colormap_binary(f1, total_density)
# heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)
heatmap(colormap, axis=nothing, frame=:none, size=(800, 800), margins=0mm)

filename = "figures/neutral_non_rescale_aniso"
savefig(filename*".pdf")
savefig(filename*".svg")
savefig(filename*".png")

In [ ]:
colormap = create_colormap(f1, total_density)
# heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)
heatmap(colormap, axis=nothing, frame=:none, size=(800, 800), margins=0mm)

In [ ]:
function initial_conditions(x, y, Lx, Ly, n_wedges=12, radius_fraction=0.03)
    # Center of the grid
    center_x, center_y = Lx / 2, Ly / 2
    
    # Radius of the circle as a fraction of the grid size
    radius = radius_fraction * min(Lx, Ly)

    # Calculate distance and angle from the center
    dist_to_center = hypot(x - center_x, y - center_y)
    angle = atan(y - center_y, x - center_x)
    
    # Normalize angle to [0, 2π)
    angle = angle < 0 ? angle + 2π : angle

    # Determine wedge index
    wedge_index = Int(floor(angle / (2π / n_wedges)))

    # Inside the circle
    if dist_to_center <= radius
        # Alternate between type 1 and type 2 based on wedge index
        if isodd(wedge_index)
            n1, n2 = 1.0, 0.0  # Type 1
        else
            n1, n2 = 0.0, 1.0  # Type 2
        end
    else
        n1, n2 = 0.0, 0.0  # Outside the circle
    end
    
    return n1, n2
end

solution = simulate_reaction_diffusion(r1=1.5, r2=1.5, a11=2.4, a21=2.38, a12=0.0, a22=0.0,
                                        D1x=0.005, D2x=0.007, D1y=0.005, D2y=0.005,
                                        Lx=200.0, Ly=200.0, Nx=600, Ny=600, t_final=220.0)

sol = solution
f1 = sol[1, :, :, end] ./ (sol[1, :, :, end] .+ sol[2, :, :, end] .+ 1e-10)  # Add a small value to avoid division by zero
total_density = sol[1, :, :, end] .+ sol[2, :, :, end]
colormap = create_colormap(f1, total_density)
# heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)
heatmap(colormap, axis=nothing, frame=:none, size=(800, 800), margins=0mm)

filename = "figures/escape_circular_colony_rd_intermediate_2"
savefig(filename*".pdf")
savefig(filename*".svg")
savefig(filename*".png")

In [ ]:
function initial_conditions(x, y, Lx, Ly, n_wedges=12, radius_fraction=0.03)
    # Center of the grid
    center_x, center_y = Lx / 2, Ly / 2
    
    # Radius of the circle as a fraction of the grid size
    radius = radius_fraction * min(Lx, Ly)

    # Calculate distance and angle from the center
    dist_to_center = hypot(x - center_x, y - center_y)
    angle = atan(y - center_y, x - center_x)
    
    # Normalize angle to [0, 2π)
    angle = angle < 0 ? angle + 2π : angle

    # Determine wedge index
    wedge_index = Int(floor(angle / (2π / n_wedges)))

    # Inside the circle
    if dist_to_center <= radius
        # Alternate between type 1 and type 2 based on wedge index
        if isodd(wedge_index)
            n1, n2 = 1.0, 0.0  # Type 1
        else
            n1, n2 = 0.0, 1.0  # Type 2
        end
    else
        n1, n2 = 0.0, 0.0  # Outside the circle
    end
    
    return n1, n2
end

solution = simulate_reaction_diffusion(r1=1.5, r2=1.5, a11=2.4, a21=2.38, a12=0.0, a22=0.0,
                                        D1x=0.005, D2x=0.007, D1y=0.005, D2y=0.005,
                                        Lx=120.0, Ly=120.0, Nx=600, Ny=600, t_final=720.0, num_times_saved=5)



In [ ]:
sol = solution
hms = []
for t_idx in 1:size(sol,4)
    println(t_idx)
    f1 = sol[1, :, :, t_idx] ./ (sol[1, :, :, t_idx] .+ sol[2, :, :, t_idx] .+ 1e-10)  # Add a small value to avoid division by zero
    total_density = sol[1, :, :, t_idx] .+ sol[2, :, :, t_idx]
    colormap = create_colormap(f1, total_density)
    # heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)
    push!(hms, heatmap(colormap, axis=nothing, frame=:none, size=(800, 800), margins=0mm))
end

plot(hms..., layout=(2,3))


filename = "figures/escape_circular_colony_rd_all"
savefig(filename*".pdf")
savefig(filename*".svg")
savefig(filename*".png")

In [ ]:
plot(hms...)

In [ ]:
function initial_conditions(x, y, Lx, Ly, n_wedges=6, radius_fraction=0.03)
    # Center of the grid
    center_x, center_y = Lx / 2, Ly / 2
    
    # Radius of the circle as a fraction of the grid size
    radius = radius_fraction * min(Lx, Ly)

    # Calculate distance and angle from the center
    dist_to_center = hypot(x - center_x, y - center_y)
    angle = atan(y - center_y, x - center_x)
    
    # Normalize angle to [0, 2π)
    angle = angle < 0 ? angle + 2π : angle

    # Determine wedge index
    wedge_index = Int(floor(angle / (2π / n_wedges)))

    # Inside the circle
    if dist_to_center <= radius
        # Alternate between type 1 and type 2 based on wedge index
        if isodd(wedge_index)
            n1, n2 = 0.0, 1.0  # Type 1
        else
            n1, n2 = 1.0, 0.0  # Type 2
        end
    else
        n1, n2 = 0.0, 0.0  # Outside the circle
    end
    
    return n1, n2
end

solution = simulate_reaction_diffusion(r1=1.5, r2=1.5, a11=0.0, a21=0.0, a12=0.0, a22=0.0,
                                        D1x=0.005, D2x=0.007, D1y=0.005, D2y=0.005,
                                        Lx=200.0, Ly=200.0, Nx=600, Ny=600, t_final=220.0)

sol = solution
f1 = sol[1, :, :, end] ./ (sol[1, :, :, end] .+ sol[2, :, :, end] .+ 1e-10)  # Add a small value to avoid division by zero
total_density = sol[1, :, :, end] .+ sol[2, :, :, end]
colormap = create_colormap(f1, total_density)
# heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)
heatmap(colormap, axis=nothing, frame=:none, size=(800, 800), margins=0mm)

filename = "figures/aniso_sector_drift"
savefig(filename*".pdf")
savefig(filename*".svg")
savefig(filename*".png")

In [ ]:
function initial_conditions(x, y, Lx, Ly, n_wedges=4, radius_fraction=0.1)
    # Center of the grid
    center_x, center_y = Lx / 2, Ly / 2
    
    # Radius of the circle as a fraction of the grid size
    radius = radius_fraction * min(Lx, Ly)

    # Calculate distance and angle from the center
    dist_to_center = abs(x - center_x) + abs(y - center_y)
    angle = atan(y - center_y, x - center_x)
    
    # Normalize angle to [0, 2π)
    angle = angle < 0 ? angle + 2π : angle

    # Determine wedge index
    wedge_index = Int(floor((angle - π/4) / (2π / n_wedges)))

    # Inside the circle
    if dist_to_center <= radius
        # Alternate between type 1 and type 2 based on wedge index
        if isodd(wedge_index)
            n1, n2 = 0.0, 1.0  # Type 1
        else
            n1, n2 = 1.0, 0.0  # Type 2
        end
    else
        n1, n2 = 0.0, 0.0  # Outside the circle
    end
    
    return n1, n2
end

solution = simulate_reaction_diffusion(r1=1.5, r2=1.5, a11=0.0, a21=0.0, a12=0.0, a22=0.0,
                                        D1x=0.005, D2x=0.005, D1y=0.005, D2y=0.005,
                                        Lx=200.0, Ly=200.0, Nx=300, Ny=300, t_final=700.0, num_times_saved=5)

sol = solution
hms = []
for t_idx in 1:size(sol,4)
    println(t_idx)
    f1 = sol[1, :, :, t_idx] ./ (sol[1, :, :, t_idx] .+ sol[2, :, :, t_idx] .+ 1e-10)  # Add a small value to avoid division by zero
    total_density = sol[1, :, :, t_idx] .+ sol[2, :, :, t_idx]
    colormap = create_colormap(f1, total_density)
    # heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)
    push!(hms, heatmap(colormap, axis=nothing, frame=:none, size=(800, 800), margins=0mm))
end

plot(hms..., layout=(2,3))

filename = "figures/diamond_neutral"
savefig(filename*".pdf")
savefig(filename*".svg")
savefig(filename*".png")

In [ ]:
function initial_conditions(x, y, Lx, Ly, n_wedges=4, radius_fraction=0.1)
    # Center of the grid
    center_x, center_y = Lx / 2, Ly / 2
    
    # Radius of the circle as a fraction of the grid size
    radius = radius_fraction * min(Lx, Ly)

    # Calculate distance and angle from the center
    dist_to_center = abs(x - center_x) + abs(y - center_y)
    angle = atan(y - center_y, x - center_x)
    
    # Normalize angle to [0, 2π)
    angle = angle < 0 ? angle + 2π : angle

    # Determine wedge index
    wedge_index = Int(floor((angle - π/4) / (2π / n_wedges)))

    # Inside the circle
    if dist_to_center <= radius
        # Alternate between type 1 and type 2 based on wedge index
        if isodd(wedge_index)
            n1, n2 = 0.0, 1.0  # Type 1
        else
            n1, n2 = 1.0, 0.0  # Type 2
        end
    else
        n1, n2 = 0.0, 0.0  # Outside the circle
    end
    
    return n1, n2
end

solution = simulate_reaction_diffusion(r1=1.5, r2=1.5, a11=0.0, a21=0.0, a12=0.0, a22=0.0,
                                        D1x=0.005, D2x=0.005, D1y=0.03, D2y=0.03,
                                        Lx=200.0, Ly=200.0, Nx=300, Ny=300, t_final=700.0, num_times_saved=5)

sol = solution
hms = []
for t_idx in 1:size(sol,4)
    println(t_idx)
    f1 = sol[1, :, :, t_idx] ./ (sol[1, :, :, t_idx] .+ sol[2, :, :, t_idx] .+ 1e-10)  # Add a small value to avoid division by zero
    total_density = sol[1, :, :, t_idx] .+ sol[2, :, :, t_idx]
    colormap = create_colormap(f1, total_density)
    # heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)
    push!(hms, heatmap(colormap, axis=nothing, frame=:none, size=(800, 800), margins=0mm))
end

plot(hms..., layout=(2,3))

filename = "figures/diamond_neutral_aniso"
savefig(filename*".pdf")
savefig(filename*".svg")
savefig(filename*".png")

In [ ]:
function initial_conditions(x, y, Lx, Ly, n_wedges=8, radius_fraction=0.03)
    # Center of the grid
    center_x, center_y = Lx / 2, Ly / 2
    
    # Radius of the circle as a fraction of the grid size
    radius = radius_fraction * min(Lx, Ly)

    # Calculate distance and angle from the center
    dist_to_center = abs(x - center_x)^2 + abs(y - center_y)^2
    angle = atan(y - center_y, x - center_x)
    
    # Normalize angle to [0, 2π)
    angle = angle < 0 ? angle + 2π : angle

    # Determine wedge index
    wedge_index = Int(floor((angle - π/4) / (2π / n_wedges)))

    # Inside the circle
    if dist_to_center <= radius
        # Alternate between type 1 and type 2 based on wedge index
        if isodd(wedge_index)
            n1, n2 = 0.0, 1.0  # Type 1
        else
            n1, n2 = 1.0, 0.0  # Type 2
        end
    else
        n1, n2 = 0.0, 0.0  # Outside the circle
    end
    
    return n1, n2
end

solution = simulate_reaction_diffusion(r1=1.5, r2=1.5, a11=0.0, a21=0.0, a12=0.0, a22=0.0,
                                        D1x=0.005, D2x=0.005, D1y=0.02, D2y=0.02,
                                        Lx=200.0, Ly=200.0, Nx=600, Ny=600, t_final=400.0)

sol = solution
f1 = sol[1, :, :, end] ./ (sol[1, :, :, end] .+ sol[2, :, :, end] .+ 1e-10)  # Add a small value to avoid division by zero
total_density = sol[1, :, :, end] .+ sol[2, :, :, end]
colormap = create_colormap(f1, total_density)
# heatmap(colormap, title="Combined Density", xlabel="x", ylabel="y", color=:auto)
heatmap(colormap, axis=nothing, frame=:none, size=(800, 800), margins=0mm)

filename = "figures/neutral_aniso_ellipse"
savefig(filename*".pdf")
savefig(filename*".svg")
savefig(filename*".png")